In [1]:
using Revise
using LogicCircuits, ProbabilisticCircuits

includet("product.jl");
includet("log.jl");
includet("support_circuit.jl");
includet("real_power.jl");
includet("integrate.jl");
includet("information.jl");
includet("quotient.jl");

Pipelines

In [2]:
function ent(pc::ProbCircuit)
    println("> Size of the input circuits: pc - $(num_edges(pc))")
    t = @elapsed log_pc = log_circuit(pc; log_prob = true)
    println("> Computing log(pc) took $(t) seconds; the result circuit has $(num_edges(log_pc)) edges")
    t = @elapsed pc_t_log_pc = product_circuit(pc, log_pc; m_log_prob = true, n_log_prob = false, compatible = true)
    println("> Computing pc * log(pc) took $(t) seconds; the result circuit has $(num_edges(pc_t_log_pc)) edges")
    t = @elapsed ent_val = -integrate_circuit(pc_t_log_pc; log_prob = false)
    println("> Computing int(pc * log(pc)) took $(t) seconds.")
    println("> The entropy of pc is $(ent_val).")
end

ent (generic function with 1 method)

In [3]:
function xent(pc1::ProbCircuit, pc2::ProbCircuit)
    println("> Size of the input circuits: pc1 - $(num_edges(pc1)), pc2 - $(num_edges(pc2))")
    t = @elapsed log_pc2 = log_circuit(pc2; log_prob = true)
    println("> Computing log(pc2) took $(t) seconds; the result circuit has $(num_edges(log_pc2)) edges")
    t = @elapsed pc1_t_log_pc2 = product_circuit(pc1, log_pc2; m_log_prob = true, n_log_prob = false, compatible = true)
    println("> Computing pc1 * log(pc2) took $(t) seconds; the result circuit has $(num_edges(pc1_t_log_pc2)) edges")
    t = @elapsed xent_val = -integrate_circuit(pc1_t_log_pc2; log_prob = false)
    println("> Computing int(pc1 * log(pc2)) took $(t) seconds.")
    println("> The cross entropy between pc1 and pc2 is $(xent_val).")
end

xent (generic function with 1 method)

In [15]:
function kld(pc1::ProbCircuit, pc2::ProbCircuit)
    println("> Size of the input circuits: pc1 - $(num_edges(pc1)), pc2 - $(num_edges(pc2))")
    t = @elapsed inv_pc2 = circuit_real_power(pc2, -1.0)
    println("> Computing 1/pc2 took $(t) seconds; the result circuit has $(num_edges(inv_pc2)) edges")
    t = @elapsed pc1_q_pc2 = product_circuit(pc1, inv_pc2; m_log_prob = true, n_log_prob = true, compatible = true)
    println("> Computing pc1/pc2 took $(t) seconds; the result circuit has $(num_edges(pc1_q_pc2)) edges")
    t = @elapsed log_pc1_q_pc2 = log_circuit(pc1_q_pc2; log_prob = false)
    println("> Computing log(pc1/pc2) took $(t) seconds; the result circuit has $(num_edges(log_pc1_q_pc2)) edges")
    t = @elapsed pc1_t_log_pc1_q_pc2 = product_circuit(pc1, log_pc1_q_pc2; m_log_prob = true, n_log_prob = false, compatible = true)
    println("> Computing pc1 * log(pc1/pc2) took $(t) seconds; the result circuit has $(num_edges(pc1_t_log_pc1_q_pc2)) edges")
    t = @elapsed kld_val = integrate_circuit(pc1_t_log_pc1_q_pc2; log_prob = false)
    println("> Computing int(pc1 * log(pc1/pc2)) took $(t) seconds.")
    println("> The KLD between pc1 and pc2 is $(kld_val).")
end

kld (generic function with 1 method)

In [5]:
function reiny_alpha_div(pc1::ProbCircuit, pc2::ProbCircuit; alpha::Float64)
    println("> Size of the input circuits: pc1 - $(num_edges(pc1)), pc2 - $(num_edges(pc2))")
    t = @elapsed powpc1 = circuit_real_power(pc1, alpha)
    println("> Computing pc1^alpha took $(t) seconds; the result circuit has $(num_edges(powpc1)) edges")
    t = @elapsed powpc2 = circuit_real_power(pc2, 1.0 - alpha)
    println("> Computing pc2^(1-alpha) took $(t) seconds; the result circuit has $(num_edges(powpc2)) edges")
    t = @elapsed powpc1_t_powpc2 = product_circuit(powpc1, powpc2; m_log_prob = true, n_log_prob = true, compatible = true)
    println("> Computing pc1^alpha * pc2^(1-alpha) took $(t) seconds; the result circuit has $(num_edges(powpc1_t_powpc2)) edges")
    t = @elapsed reiny_val = log(integrate_circuit(powpc1_t_powpc2; log_prob = false)) / (1.0 - alpha)
    println("> Computing int(pc1^alpha * pc2^(1-alpha)) took $(t) seconds.")
    println("> The Reiny's Alpha Divergence between pc1 and pc2 is $(reiny_val).")
end

reiny_alpha_div (generic function with 1 method)

Load PCs

In [6]:
pc1 = load_prob_circuit("pcs/plants_100.psdd");
pc2 = load_prob_circuit("pcs/plants_500.psdd");

In [11]:
ent(pc1)

> Size of the input circuits: pc - 4854
> Computing log(pc) took 0.021104033 seconds; the result circuit has 17742 edges
> Computing pc * log(pc) took 0.073178862 seconds; the result circuit has 17742 edges
> Computing int(pc * log(pc)) took 0.003401263 seconds.
> The entropy of pc is 15.3851550327636.


In [12]:
xent(pc1, pc2)

> Size of the input circuits: pc1 - 4854, pc2 - 63562
> Computing log(pc2) took 0.175586626 seconds; the result circuit has 232989 edges
> Computing pc1 * log(pc2) took 1.159245596 seconds; the result circuit has 233045 edges
> Computing int(pc1 * log(pc2)) took 0.169091174 seconds.
> The cross entropy between pc1 and pc2 is 17.385056426813467.


In [17]:
kld(pc1, pc2)

> Size of the input circuits: pc1 - 4854, pc2 - 63562
> Computing 1/pc2 took 0.031285941 seconds; the result circuit has 63562 edges
> Computing pc1/pc2 took 0.362805622 seconds; the result circuit has 63618 edges
> Computing log(pc1/pc2) took 0.162056442 seconds; the result circuit has 233157 edges
> Computing pc1 * log(pc1/pc2) took 1.340772199 seconds; the result circuit has 233157 edges
> Computing int(pc1 * log(pc1/pc2)) took 0.062308202 seconds.
> The KLD between pc1 and pc2 is 1.9999013940498658.


In [14]:
reiny_alpha_div(pc1, pc2; alpha = 1.5)

> Size of the input circuits: pc1 - 4854, pc2 - 63562
> Computing pc1^alpha took 0.002705348 seconds; the result circuit has 4854 edges
> Computing pc2^(1-alpha) took 0.023019769 seconds; the result circuit has 63562 edges
> Computing pc1^alpha * pc2^(1-alpha) took 0.310346047 seconds; the result circuit has 63618 edges
> Computing int(pc1^alpha * pc2^(1-alpha)) took 0.018752003 seconds.
> The Reiny's Alpha Divergence between pc1 and pc2 is -6.009454249699177.
